In [3]:
import os

import matplotlib.pyplot as plt
import pandas as pd
# import seaborn as sns
import numpy as np
from sklearn.manifold import TSNE
import pickle
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import confusion_matrix

In [4]:
def load_files(path):
    df = pd.read_csv(path+'clustering_results.csv')
    
    domain = df['predictions'].values
    
    vec_d = df['vec_d_labels'].values
    vec_y = df['vec_y_labels'].values
    img_locs = df['image_id_labels'].values

    Z = np.load(path + "Z_space.npy")
    model_name = path.split('_')[-1]

    #args = np.loadtxt(path + "arguments.txt", dtype=str, usecols=0)  # , delimiter = '\n')
    #img_locs = np.loadtxt(path + "img_id.txt", dtype=str)
    with open(os.path.join(path, 'commandline_arguments.p'), 'rb') as file:
        command_line = pickle.load(file)
    
    return domain, vec_d, vec_y, img_locs, model_name, df, command_line

In [5]:
ex1 = '2023-09-28 12:47:42.403147_weah_sdcn/'
ex2 = '2023-10-03 15:29:14.413030_weah_dec/'
ex2= '2023-10-10 11:09:07.411289_weah_dec/'
ex2='2023-10-15 16:52:23.823748_weah_sdcn/'

In [6]:
domain1, vec_d1, vec_y1, img_locs1, model_name1, results_df1, command_line1 = load_files(ex1)
domain2, vec_d2, vec_y2, img_locs2, model_name2, results_df2, command_line2 = load_files(ex2)

In [7]:
def correlation_with_labels(domain, labels, model_name='unknown'):
    print('Model', model_name)
    domain=domain-1
    print('Unique labels:', np.unique(labels),'unique domains:', np.unique(domain))
    if len(np.unique(labels))>len(np.unique(domain)):
        N = len(np.unique(labels))
    else:
        N = len(np.unique(domain))
    cost = np.zeros((N,N))
    
    cluster_pred_scalar = domain
    cluster_true_scalar = labels
    cost = cost - confusion_matrix(cluster_pred_scalar, cluster_true_scalar)

    row_ind, col_ind = linear_sum_assignment(cost)

    conf_mat = (-1)*cost[:, col_ind]

    acc_d = np.diag(conf_mat).sum() / conf_mat.sum()
    print("Confusion Matrix:")
    print(conf_mat)
    print('Correlation with the labels', acc_d)
    remapped_domain = np.array([col_ind[np.unique(domain).searchsorted(d)] for d in domain])

    return remapped_domain
    

In [8]:
ann1 = [int(img_loc.split('/')[-1].split('_')[2]) for img_loc in img_locs1]
ann2 = [int(img_loc.split('/')[-1].split('_')[2]) for img_loc in img_locs2]


In [30]:
img_locs1[0]

'../../WashU_with_coord/combined_training_with_coords/APERIO-138-0/aperio-138-0_anno_2_reg_1CAH_coord_47744_36866_patchnumber_0_xy1024_918.png'

In [51]:
reg_dict = {'H': 0, 'a':1, 'e':2, 'n':3}
reg1 = [reg_dict[img_loc.split('/')[-1].split('_')[4][-1]] for img_loc in img_locs1]

reg2 = [reg_dict[img_loc.split('/')[-1].split('_')[4][-1]] for img_loc in img_locs2]


In [48]:
np.unique(reg1)

array(['H', 'a', 'e'], dtype='<U1')

In [49]:
remapped_domain1 = correlation_with_labels(domain1, ann1,command_line1.aname)

Model sdcn
Unique labels: [1 2 3] unique domains: [0 1 2 3 4 5]
Confusion Matrix:
[[-0.0000e+00  4.2700e+02 -0.0000e+00  2.0900e+02 -0.0000e+00  1.2500e+02]
 [-0.0000e+00  1.9911e+04 -0.0000e+00  7.6700e+03 -0.0000e+00  5.9900e+02]
 [-0.0000e+00 -0.0000e+00 -0.0000e+00 -0.0000e+00 -0.0000e+00  5.0000e+00]
 [-0.0000e+00  1.6464e+04 -0.0000e+00  1.0898e+04 -0.0000e+00  1.1000e+01]
 [-0.0000e+00  1.3500e+02 -0.0000e+00  1.0000e+01 -0.0000e+00  1.3000e+01]
 [-0.0000e+00  2.6300e+02 -0.0000e+00  4.1300e+02 -0.0000e+00  4.4700e+02]]
Correlation with the labels 0.5426388888888889


In [19]:
remapped_domain2 = correlation_with_labels(domain2, ann2, command_line2.aname)

Model sdcn
Unique labels: [1 2 3] unique domains: [0 1 2 3 4 5]
Confusion Matrix:
[[   -0.   256.   725.  1102.    -0.    -0.]
 [   -0.  2178.  1857.  3896.    -0.    -0.]
 [   -0.   188.   767.   656.    -0.    -0.]
 [   -0.  8424.  8707. 28336.    -0.    -0.]
 [   -0.    69.   174.   279.    -0.    -0.]
 [   -0.   217.   221.   448.    -0.    -0.]]
Correlation with the labels 0.5347179487179488


In [52]:
remapped_domain1 = correlation_with_labels(domain1, reg1,command_line1.aname)

Model sdcn
Unique labels: [0 1 2] unique domains: [0 1 2 3 4 5]
Confusion Matrix:
[[-0.0000e+00  4.2700e+02 -0.0000e+00  2.0900e+02 -0.0000e+00  1.2500e+02]
 [-0.0000e+00  1.9911e+04 -0.0000e+00  7.6700e+03 -0.0000e+00  5.9900e+02]
 [-0.0000e+00 -0.0000e+00 -0.0000e+00 -0.0000e+00 -0.0000e+00  5.0000e+00]
 [-0.0000e+00  1.6464e+04 -0.0000e+00  1.0898e+04 -0.0000e+00  1.1000e+01]
 [-0.0000e+00  1.3500e+02 -0.0000e+00  1.0000e+01 -0.0000e+00  1.3000e+01]
 [-0.0000e+00  2.6300e+02 -0.0000e+00  4.1300e+02 -0.0000e+00  4.4700e+02]]
Correlation with the labels 0.5426388888888889


In [54]:
remapped_domain2 = correlation_with_labels(domain2, reg2, command_line2.aname)

Model sdcn
Unique labels: [0 1 2 3] unique domains: [0 1 2 3 4 5]
Confusion Matrix:
[[   70.   186.   725.  1102.    -0.    -0.]
 [  484.  1694.  1857.  3896.    -0.    -0.]
 [   -0.   188.   767.   656.    -0.    -0.]
 [ 1283.  7141.  8707. 28336.    -0.    -0.]
 [   -0.    69.   174.   279.    -0.    -0.]
 [   38.   179.   221.   448.    -0.    -0.]]
Correlation with the labels 0.5276410256410257


In [20]:
from sklearn.metrics import cohen_kappa_score
kappa = cohen_kappa_score(remapped_domain1, ann1)
print("Cohen's Kappa SDCN:", kappa)


Cohen's Kappa SDCN: 0.1292013312866832


In [21]:
kappa = cohen_kappa_score(remapped_domain2, ann2)
print("Cohen's Kappa DEC:", kappa)

Cohen's Kappa DEC: 0.0816269368341096


In [22]:
    # print(confusion_matrix(remapped_domain, labels))
    # remapped_confusion=confusion_matrix(remapped_domain, labels)
    # print(np.diag(remapped_confusion).sum() / remapped_confusion.sum())

In [23]:
def subject_info(img_locs,subjects,domain, N):
    sub_img = []
    sub_domain = []

    for i, (img) in enumerate(img_locs):
        if subjects[N] in img:
            sub_img.append(img)
            sub_domain.append(domain[i])
    patch_x = [int(img.split('/')[-1].split('_')[-6]) for img in sub_img]
    patch_y = [int(img.split('/')[-1].split('_')[-5]) for img in sub_img]

    x_coords = [int(img.split('/')[-1].split('_')[-2][2:]) for img in sub_img]
    y_coords = [int(img.split('/')[-1].split('_')[-1].split('.')[0]) for img in sub_img]
    return sub_img,sub_domain, patch_x, patch_y, x_coords, y_coords

In [24]:
def prob_neiboring_domain_sim(x_coords, y_coords,sub_domain):
    distance =np.zeros((len(x_coords), len(x_coords)))
    domain_mat =np.zeros((len(x_coords), len(x_coords)))
    for i in range(0, len(x_coords)):
        for j in range(0, len(x_coords)):
            distance[i, j]=np.sqrt((x_coords[i]-x_coords[j])**2+(y_coords[i]-y_coords[j])**2)
            if sub_domain[i]==sub_domain[j]:
                domain_mat[i, j]=1
    probability=[]
    for k in range(0, len(distance)):
        row_distance = distance[k, :] 
        min_distance_idx = np.argmin(row_distance)
        row_distance = np.delete(row_distance,min_distance_idx)
        min_distance_idx = np.argmin(row_distance)
        row_domain = domain_mat[k, min_distance_idx]
        probability.append(row_domain)
    return sum(probability)/len(probability)

In [25]:
subjects1 = list(set([l.split('/')[4] for l in img_locs1]))
subjects2 = list(set([l.split('/')[4] for l in img_locs2]))

In [26]:
P = []
for N, sub in enumerate(subjects1):
    sub_img,sub_domain, patch_x, patch_y, x_coords, y_coords = subject_info(img_locs1,subjects1,domain1, N)
    
    p_sub = prob_neiboring_domain_sim(x_coords, y_coords,sub_domain)
    P.append(p_sub)
print(np.mean(P))

0.8019270833333334


In [27]:
P = []
for N, sub in enumerate(subjects2):
    sub_img,sub_domain, patch_x, patch_y, x_coords, y_coords = subject_info(img_locs2,subjects2, domain2, N)
    p_sub = prob_neiboring_domain_sim(x_coords, y_coords,sub_domain)
    P.append(p_sub)
print(np.mean(P))

0.7776239316239317
